## Fake news classification with 90% accuracy uisng LSTM

In [1]:
import pandas as pd
import numpy as np
import nltk

In [2]:
df = pd.read_csv('D:/ML_learning/Fake_News_Classification/fake-news/train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [3]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [4]:
df = df.dropna()

In [5]:
## Dependent and Independent features

X= df.drop('label', axis=1)
y = df['label']

In [6]:
X.shape

(18285, 4)

In [7]:
y.shape

(18285,)

In [18]:
## Import all the libraries for LSTM
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [12]:
# copy the data
news = df.copy()

In [13]:
news.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [14]:
news.reset_index(inplace=True)

In [15]:
news['title'][2]

'Why the Truth Might Get You Fired'

In [16]:
## Now we will clean the data
import re
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()

corpus = []

for i in range(0, len(news)):
    review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [28]:
corpus[0]

'hous dem aid even see comey letter jason chaffetz tweet'

In [19]:
## Once we have all the text preprocessed, we will convert the text using one hot.
### Vocabulary size
voc_size=5000

onehot_rep = [one_hot(words, voc_size) for words in corpus]
onehot_rep

[[1978, 158, 2657, 298, 2724, 201, 3050, 2326, 1059, 4957],
 [4375, 1038, 2147, 1290, 3176, 3108, 4200],
 [1166, 246, 1878, 1091],
 [4301, 2199, 3027, 1497, 3167, 528],
 [37, 3176, 542, 3483, 645, 2590, 3176, 3277, 1889, 4955],
 [2372,
  2413,
  2956,
  3556,
  3787,
  1928,
  4402,
  1481,
  756,
  496,
  3981,
  1910,
  2817,
  1396,
  4200],
 [3269, 4800, 1203, 3578, 4758, 3095, 3198, 4879, 1316, 2797, 2339],
 [3064, 3420, 4369, 3365, 286, 30, 1928, 2442, 1316, 2797, 2339],
 [3600, 3216, 990, 4564, 2707, 2055, 4035, 2838, 1928, 2578],
 [4660, 905, 1218, 3902, 330, 2373, 1966, 3507],
 [2548, 3918, 1746, 4040, 2852, 1648, 4792, 4039, 1732, 628, 3433],
 [1497, 479, 2724, 2055, 1928, 286],
 [1082, 4054, 2829, 4585, 4456, 3933, 1004, 3977, 1129],
 [3684, 4873, 1008, 2758, 3632, 485, 4780, 1316, 2797, 2339],
 [3524, 1895, 1952, 4304, 809, 1316, 2797, 2339],
 [2769, 619, 192, 2340, 1292, 4349, 4659, 1848, 2774, 3119],
 [4054, 3629, 1038],
 [2432, 2117, 3996, 2016, 1928, 287, 4171, 4200],
 

In [27]:
## Now pass this onehot endcoding output to embedding layer

## Before that make all the sentences of equal length
sentence_len = 20 # maximum sentence length to make all sentences of equal size
embeded_doc = pad_sequences(onehot_rep, maxlen= sentence_len, padding='pre')
embeded_doc[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1978,
        158, 2657,  298, 2724,  201, 3050, 2326, 1059, 4957])

In [28]:
## Now creating the model

vector_size = 40
model = Sequential()
model.add(Embedding(voc_size, vector_size, input_length = sentence_len))
model.add(LSTM(100))
model.add(Dense(1, activation= 'sigmoid'))
model.compile(loss='binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [29]:
## Now convert the vectors as array
import numpy as np
X_final=np.array(embeded_doc)
y_final=np.array(y)

In [32]:
## split data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.30, random_state=42)

In [33]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 2s 11ms/step - loss: 0.3419 - accuracy: 0.8365 - val_loss: 0.2020 - val_accuracy: 0.9140
Epoch 2/10
200/200 [==============================] - 2s 9ms/step - loss: 0.1432 - accuracy: 0.9446 - val_loss: 0.2063 - val_accuracy: 0.9209
Epoch 3/10
200/200 [==============================] - 2s 9ms/step - loss: 0.1010 - accuracy: 0.9623 - val_loss: 0.2100 - val_accuracy: 0.9129
Epoch 4/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0746 - accuracy: 0.9740 - val_loss: 0.2412 - val_accuracy: 0.9138
Epoch 5/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0508 - accuracy: 0.9840 - val_loss: 0.2957 - val_accuracy: 0.9101
Epoch 6/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0333 - accuracy: 0.9895 - val_loss: 0.3703 - val_accuracy: 0.9127
Epoch 7/10
200/200 [==============================] - 2s 9ms/step - loss: 0.0214 - accuracy: 0.9942 - val_loss: 0.4489 - val_accuracy: 0.9112
Epoch

In [37]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sentence_len))
model.add(Dropout(0.3))
model.add(LSTM(50))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [38]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
200/200 [==============================] - 2s 9ms/step - loss: 0.3516 - accuracy: 0.8289 - val_loss: 0.2024 - val_accuracy: 0.9110
Epoch 2/10
200/200 [==============================] - 1s 6ms/step - loss: 0.1539 - accuracy: 0.9398 - val_loss: 0.1957 - val_accuracy: 0.9202
Epoch 3/10
200/200 [==============================] - 1s 6ms/step - loss: 0.1138 - accuracy: 0.9569 - val_loss: 0.2098 - val_accuracy: 0.9127
Epoch 4/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0901 - accuracy: 0.9671 - val_loss: 0.2500 - val_accuracy: 0.9141
Epoch 5/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0726 - accuracy: 0.9752 - val_loss: 0.2601 - val_accuracy: 0.9036
Epoch 6/10
200/200 [==============================] - 1s 6ms/step - loss: 0.0617 - accuracy: 0.9791 - val_loss: 0.2743 - val_accuracy: 0.9083
Epoch 7/10
200/200 [==============================] - 1s 7ms/step - loss: 0.0542 - accuracy: 0.9803 - val_loss: 0.3137 - val_accuracy: 0.9045
Epoch 

In [39]:
# Prediction
y_pred=model.predict_classes(X_test)

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


In [42]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[2851,  256],
       [ 271, 2108]], dtype=int64)

In [43]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9039372949325556